# Example: Fun with Iterative Linear Algebraic Equation (LAE) Solvers
In this example, let's explore the properties and performance of three iterative linear algebraic solvers: the Jacobi method, the Gauss-Seidel method, and the Successive Over-Relaxation (SOR) method. 

> __Learning Objectives:__
>
> By the end of this activity, you should be able to:
> * __Random System Generation__: Generate random diagonally dominant system matrices $\mathbf{A}$ and right-hand-side vectors $\mathbf{b}$ of a specified dimension. We'll use these test matrices for benchmarking studies later in the activity.
> * __Solve a random square system__: Solve the LAEs using the Jacobi, Gauss-Seidel and SOR methods. In this task, we'll solve our system of random linear algebraic equations using the [Jacobi](https://en.wikipedia.org/wiki/Jacobi_method), [Gauss-Seidel](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method), and [SOR](https://en.wikipedia.org/wiki/Successive_over-relaxation) methods.
> * __Benchmark__: In this task, we'll compare the runtime performance of the different iterative approaches against the built-in method implemented by [the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/) included with Julia [using the `BenchmarkTools.jl` package.](https://github.com/JuliaCI/BenchmarkTools.jl)

This should be fun, so let's go!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's setup your environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl"));

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl), check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types and data used in this material. 

___

## Task 1: Build random diagonally dominant matrices
In this task, we'll generate a random system matrix $\mathbf{A}$ that is diagonally dominant and a random right-hand side vector $\mathbf{b}$. 


> __Diagonal dominance__ is a matrix property where the absolute value of the diagonal element of each row is greater than the sum of the absolute values of the other elements in that row.  Diagonal dominance is a sufficient (but not necessary) condition for convergence of iterative methods, however, this condition says nothing about the rate of convergence.

A diagonally dominant system matrix $\mathbf{A}$ has the feature:
$$
\begin{equation*}
\sum_{j=1,i}^{n}\lvert{a_{ij}}\rvert<\lvert{a_{ii}}\rvert\qquad\forall{i}
\end{equation*}
$$


Let's start by specifying how many rows we have in the _square_ system matrix $\mathbf{A}$ in the `number_of_rows::Int64` variable:

In [2]:
number_of_rows = 100; # increase the number of rows for larger problem size

Next, we generate a $n\times{n}$ random system matrix $\mathbf{A}$ and a $n\times{1}$ random vector $\mathbf{b}$, [using the `randn(...)` method](https://docs.julialang.org/en/v1/stdlib/Random/#Base.randn). We add some extra to the diagonal elements of the test system matrix $\mathbf{A}$ to ensure diagonal dominance.

In [3]:
A,b = let

    # initialize -
    ϵ = 10.0; # extra stuff that we add to diagonal elements
    A = (1/ϵ)*rand(number_of_rows, number_of_rows) .+ ϵ*diagm(rand(number_of_rows)); # Interesting!
    b = ϵ*rand(number_of_rows);

    A,b
end;

What does $\mathbf{A}$ look like?

In [4]:
A

100×100 Matrix{Float64}:
 5.80266     0.0854931   0.0333791   …  0.0525582   0.096487    0.0368234
 0.017033    8.42437     0.0487222      0.064586    0.0598619   0.0980186
 0.0623465   0.0919301   9.39847        0.078199    0.00483644  0.0340437
 0.0247749   0.0385506   0.0795655      0.0219072   0.0756997   0.0494401
 0.0474572   0.0782414   0.00584511     0.0923643   0.0805831   0.0266333
 0.0259679   0.0717777   0.0240308   …  0.0318752   0.0237737   0.0559262
 0.0863729   0.00915359  0.0420482      0.083373    0.00947534  0.0331111
 0.0804549   0.0467689   0.0397823      0.0143953   0.0928532   0.0627972
 0.0763584   0.0415765   0.0170317      0.0614015   0.0240338   0.0162323
 0.00983324  0.0810885   0.0456119      0.0598723   0.0475709   0.0428205
 ⋮                                   ⋱                          
 0.0958606   0.0709765   0.0102805      0.0711612   0.0853803   0.0341746
 0.0684312   0.0294139   0.0673111      0.0650902   0.0240292   0.0407567
 0.0726067   0.0074649

### Check: Is the system matrix $\mathbf{A}$ strictly diagonally dominant?
Before we continue, let's verify the randomly generated system matrix $\mathbf{A}$ is actually diagonally dominant.
> __Test:__ We'll compute the sum of the absolute values of each row (excluding the diagonal element), and compare it to the absolute value of the diagonal element in that row. If the sum of the absolute values of the non-diagonal elements is less than the absolute value of the diagonal element, then the matrix is diagonally dominant.

Is the matrix $\mathbf{A}$ strictly diagonally dominant?

In [5]:
ddcondition = let
    
    # initialize -
    ddcondition = Array{Bool,1}(undef, number_of_rows);

    # let's check each row
    for i ∈ 1:number_of_rows
        aii = abs(A[i,i]);
        σ = 0.0;
        for j ∈ 1:number_of_rows
            if (i ≠ j)
                σ += abs(A[i,j]);
            end
        end
        ddcondition[i] = (aii > σ) ? true : false; # ternary operator, nice!
    end

    ddcondition
end;

### Understanding the Diagonal Dominance Check

The `ddcondition` array contains boolean values indicating whether each row of our system matrix $\mathbf{A}$ satisfies the diagonal dominance condition. Each element `ddcondition[i]` is `true` if the absolute value of the diagonal element in row `i` is greater than the sum of the absolute values of all other elements in that row, and `false` otherwise.

> __Test:__ The assertion `@assert any(ddcondition)` checks that at least one row (and hopefully all rows) satisfies the diagonal dominance condition. If the condition fails, a warning is issued indicating that the matrix may not be suitable for iterative methods, as convergence is not guaranteed.

For diagonally dominant matrices, iterative methods like Jacobi, Gauss-Seidel, and SOR are guaranteed to converge, making them reliable choices for solving linear systems.

In [6]:
try
    @assert any(ddcondition)
catch
    @warn "Diagonal dominance condition not satisfied"
end

__No warnings?__ Great, let's move on to solving the system of linear algebraic equations (LAEs) using one of our iterative methods.

___

## Task 2: Let's test our iterative solvers
In this task, let's test the correctness of our iterative linear solver implementations. 

> __Idea:__ We'll solve the system of linear algebraic equations $\mathbf{A}\mathbf{x}=\mathbf{b}$ using the Jacobi, Gauss-Seidel, and SOR methods, and compare the results to the solution obtained using [Julia's built-in backslash operator (`\`)](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Base.:\\-Tuple{AbstractMatrix,%20AbstractVecOrMat}), which uses a polyalgorithm approach, where the choice of a specific method depends on the problem structure and size.

First, let's use the backslash operator to obtain the Julia solution (which we will use as a reference). We'll store the reference solution the `julia_solution::Array{Float64,1}` variable.

In [7]:
julia_solution = A\b; # Julia solution

Next, let's set up a code block to compute the solutions using the Jacobi, Gauss-Seidel, and SOR methods. We'll compare the results to the Julia solution we got earlier.


> __Explanation:__ In the following code block, we'll solve our linear system $\mathbf{A}\mathbf{x}=\mathbf{b}$ using one of our iterative methods. 
> 
> * The code sets up the algorithm parameters including maximum iterations, convergence tolerance, and initial guess, then calls our custom solver implementation. 
> * The solver returns a dictionary (`our_solution_archive::Dict{Int64, Vector{Float64}}`) that stores the solution vector (value) at each iteration (key), allowing us to track the convergence behavior and retrieve the final converged solution. 
> * You can switch between different iterative methods (Jacobi, Gauss-Seidel, or SOR) by changing the `algorithm` parameter.

So what do we get?

In [8]:
our_solution_archive = let

    # initialize -
    maximum_number_of_iterations = 1000;
    tolerance = 1e-12;
    algorithm = GaussSeidelMethod(); # change this to GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 1.0; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

    # call the solve method with the appropriate parameters -
    x = VLDataScienceMachineLearningPackage.solve(A, b, xₒ; algorithm = algorithm, 
        maxiterations = maximum_number_of_iterations, 
        ϵ = tolerance, 
        ω = ω # only used for SOR
    );

    x; # return the solution
end;

In [9]:
our_solution_archive

Dict{Int64, Vector{Float64}} with 35 entries:
  5  => [0.711777, 0.55777, 0.587827, 0.91378, -0.468157, -0.400255, 3.46543, 0…
  16 => [0.705763, 0.551001, 0.582942, 0.909248, -0.472284, -0.4014, 3.44547, 0…
  20 => [0.705762, 0.551001, 0.582942, 0.909248, -0.472284, -0.4014, 3.44547, 0…
  12 => [0.705768, 0.551011, 0.582947, 0.909263, -0.472278, -0.401397, 3.4455, …
  24 => [0.705762, 0.551001, 0.582942, 0.909248, -0.472284, -0.4014, 3.44547, 0…
  28 => [0.705762, 0.551001, 0.582942, 0.909248, -0.472284, -0.4014, 3.44547, 0…
  8  => [0.705436, 0.550633, 0.582674, 0.90903, -0.472495, -0.401454, 3.44442, …
  17 => [0.705763, 0.551001, 0.582942, 0.909248, -0.472284, -0.4014, 3.44547, 0…
  30 => [0.705762, 0.551001, 0.582942, 0.909248, -0.472284, -0.4014, 3.44547, 0…
  1  => [1.24058, 0.932213, 0.79183, 1.4298, 0.291202, -0.00711807, 5.62651, 1.…
  19 => [0.705762, 0.551001, 0.582942, 0.909248, -0.472284, -0.4014, 3.44547, 0…
  0  => [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1  …  0

### Check: Solution consistency
Before we proceed, let's check the difference between the solution produced by Julia and the solution we computed using our iterative solution techniques. 

Let's compute the magnitude of the residual vector between our solution and the Julia solution using a [norm function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.norm) exported by [the `LinearAlgebra.jl` package included with the standard library](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#man-linalg).

> __Vector norm__
>
> A function $|\cdot|:\mathbb{K}^{n}\to\;[0,\infty)$ ($\mathbb{K}=\mathbb{R}$ or $\mathbb{C}$) is a **norm** if, for all $u,v\in\mathbb{K}^n$ and $\alpha\in\mathbb{K}$,
>
> 1. **Definiteness:** $|v|\ge 0$ and $|v|=0 \iff v=0$.
> 2. **Homogeneity:** $|\alpha v|=|\alpha|\times|v|$.
> 3. **Triangle inequality:** $|u+v|\le |u|+|v|$.

Here, we will use a particular norm, the p-norm:

> **Definition ($p$-norm / $\ell_p$ norm).**
> 
> Let $\mathbb{K}\in{\mathbb{R},\mathbb{C}}$ and $x=(x_1,\ldots,x_n)\in\mathbb{K}^n$. Then, for $1\le p<\infty$ the p-norm is given by:
> $$
> \|x\|_{p}=\Big(\sum_{i=1}^n |x_i|^{\,p}\Big)^{1/p}.
> $$
> and for $p=\infty$:
> $$
> \|x\|_{\infty}=\max_{1\le i\le n}|x_i|.
> $$

Let's keep it simple and use the $p = 2$ norm for our residual calculation (the default).

In [10]:
let

    # initialize -
    atol = 1e-8; # absolute tolerance
    i = keys(our_solution_archive) |> collect |> sort |> last; # get the last key (the last iteration)
    last_solution = our_solution_archive[i]; # get the last solution (converged solution)

    r = last_solution - julia_solution; 
    norm_r = norm(r) # 2-norm of the residual

    # check: our solution vs Julia solution, should be approximately equal
    @assert isapprox(norm_r, 0, atol=atol)
end

If we don't have any error, then our iterative solver is working correctly! But how well does it perform compared to Julia's built-in solver? Let's check that question out next.
___

## Task 3: Let's compare the performance of our solvers to the built-in Julia solvers
In this task, we'll benchmark our iterative solvers against Julia's built-in solvers to see how they perform on the same problem.

>__Expectation:__ We expect Julia's built-in solvers to be highly optimized and potentially __much__ faster than our iterative solvers, especially for larger problem sizes. However, our solvers may still perform competitively for smaller problems or specific cases.

Let's start with Julia's built-in solvers.

In [11]:
@benchmark $A\$b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  48.000 μs …  2.693 ms  ┊ GC (min … max): 0.00% … 96.83%
 Time  (median):     48.667 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.851 μs ± 44.099 μs  ┊ GC (mean ± σ):  2.56% ±  3.55%

  ▄██▆▃▁▁▂▁▁▁▁▂▁▁▁                                            ▂
  ██████████████████████▇▇▇▆▇▇▆▅▅▆▅▅▅▄▆▅▅▆▆▄▅▅▅▅▅▅▅▄▅▅▄▅▅▄▆▆▅ █
  48 μs        Histogram: log(frequency) by time      64.4 μs <

 Memory estimate: 81.89 KiB, allocs estimate: 7.

Now, let's benchmark our iterative solvers against Julia's built-in solvers to see how they perform on the same problem.

In [12]:
let
    # initialize -
    maximum_number_of_iterations = 10000;
    tolerance = 1e-8;
    algorithm = GaussSeidelMethod(); # change this to GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 0.6; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

     # call the solve method with the appropriate parameters -
    @benchmark VLDataScienceMachineLearningPackage.solve($A, $b, $xₒ; algorithm = $algorithm, 
        maxiterations = $maximum_number_of_iterations, 
        ϵ = $tolerance, 
        ω = $ω # only used for SOR
    );
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  135.084 μs …  2.474 ms  ┊ GC (min … max): 0.00% … 29.09%
 Time  (median):     139.583 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   158.992 μs ± 93.334 μs  ┊ GC (mean ± σ):  7.42% ± 11.79%

  █▄▂▃▁▁                                                   ▁   ▁
  █████████▇█▇▆▆▆▆▅▅▆▅▅▄▅▅▄▅▅▄▄▁▃▄▄▅▄▁▁▄▃▄▄▃▁▄▃▃▃▁▁▁▁▃▃▃▃▅▇██▇ █
  135 μs        Histogram: log(frequency) by time       558 μs <

 Memory estimate: 598.06 KiB, allocs estimate: 281.

### What did we see?
The Julia implementation is efficient, even for larger problem sizes. The iterative solvers we implemented perform comparably (similar order of magnitude median runtime) to the built-in solvers, but there are some notable differences in memory usage and convergence behavior.

> __Differences:__ Our iterative solvers consistently use significantly more memory than the built-in solvers, particularly for larger problem sizes. Furthermore, our solvers have more allocations. This is likely due to the way we store and manipulate intermediate results (e.g., for informational purposes we keep all intermediate solutions). Occasionally, our solvers may diverge or exhibit slower convergence compared to the built-in methods.

Another data point in the ongoing story that is __buy versus build__. Unless you need some specialized behavior, it's almost always better to use established libraries and frameworks rather than building your own solutions from scratch!

___